In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [39]:
trans = transforms.Compose([

    transforms.Resize(size=(256,256)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5]),

])

In [40]:
train_dataset=datasets.ImageFolder('C:/Users/visha/Downloads/archive/chest_xray/train',transform=trans)
val_dataset = datasets.ImageFolder('C:/Users/visha/Downloads/archive/chest_xray/test',transform=trans)

In [51]:
print(train_dataset.class_to_idx)

{'NORMAL': 0, 'PNEUMONIA': 1}


In [41]:
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(train_dataset,batch_size =64,shuffle=False)

In [42]:
device = torch.device('cuda' if (torch.cuda.is_available()==True) else 'cpu')
device

device(type='cuda')

In [43]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.conv1 = nn.Conv2d(1, 64, 7, stride=2, padding=3, bias=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,2)

In [44]:
model = model.to(device)

In [45]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [46]:
no_of_epochs = 5
for e in range(no_of_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_data_points = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        loss = loss_function(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_data_points += labels.size(0)
        total_loss += loss.item()

        predicted_vals = torch.max(output, dim=1)[1]
        correct_preds += (predicted_vals == labels).sum().item()

    epoch_accuracy = 100 * correct_preds / total_data_points
    epoch_loss = total_loss / len(train_loader)
    print(f'Epoch {e+1}/{no_of_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.2f}%')

Epoch 1/5 | Loss: 0.1257 | Accuracy: 94.80%
Epoch 2/5 | Loss: 0.0509 | Accuracy: 98.20%
Epoch 3/5 | Loss: 0.0427 | Accuracy: 98.45%
Epoch 4/5 | Loss: 0.0278 | Accuracy: 99.00%
Epoch 5/5 | Loss: 0.0257 | Accuracy: 98.96%


In [47]:
model.eval()
val_correct_preds = 0
val_total_data_points = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels_batch in test_loader:
        images, labels_batch = images.to(device), labels_batch.to(device)

        val_total_data_points += images.size(0)
        output = model(images)
        val_predicted_vals = torch.max(output, dim=1)[1]
        val_correct_preds += (val_predicted_vals == labels_batch).sum().item()

        all_preds.extend(val_predicted_vals.cpu().numpy())
        all_labels.extend(labels_batch.cpu().numpy())

val_accuracy = 100 * val_correct_preds / val_total_data_points
print(f'Validation Accuracy = {val_accuracy:.2f}%')

Validation Accuracy = 98.75%


In [52]:
torch.save(model.state_dict(),"resnet weights.pth")

In [53]:
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(all_labels, all_preds)
cr = classification_report(all_labels,all_preds)
print("--- Confusion Matrix ---")
print(cm)
print("---Classification Report---")
print(cr)


--- Confusion Matrix ---
[[1314   27]
 [  38 3837]]
---Classification Report---
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1341
           1       0.99      0.99      0.99      3875

    accuracy                           0.99      5216
   macro avg       0.98      0.99      0.98      5216
weighted avg       0.99      0.99      0.99      5216



In [57]:
#just testing validating one more time with a small dataset
val_dataset = datasets.ImageFolder('C:/Users/visha/Downloads/archive/chest_xray/val',transform=trans)
val_loader = DataLoader(val_dataset,batch_size=1,shuffle=False)
#set the model to eval
model.eval()
all_preds =[]
all_labels = []
with torch.no_grad():
    for images,labels in val_loader:
        images,labels = images.to(device),labels.to(images)
        output = model(images)
        predicted_vals = torch.max(output,dim =1 )[1]
        all_preds.extend(predicted_vals.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
print(confusion_matrix(all_preds,all_labels))


[[7 0]
 [1 8]]


In [59]:
from PIL import Image

model.eval()
filepath = 'img.png'

image = Image.open(filepath)
image = trans(image)
image = image.unsqueeze(0)

image = image.to(device)
output = model(image)
predicted = torch.max(output,dim = 1)[1].item()
print(predicted)

1


In [1]:
print(2+True)

3
